In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
import math
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

print("Done!")


2024-07-01 13:34:25.698129: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-01 13:34:25.704459: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-01 13:34:25.763476: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-01 13:34:26.965329: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Done!


In [2]:
training_data=pd.read_csv("training_data_1.csv")
real_data=pd.read_csv("real_data_1.csv")

noisy_test=pd.read_csv("training_data_test.csv")
real_test=pd.read_csv("real_data_test.csv")

print("Done!")

Done!


In [3]:
Params= 100,304,16
Params1=100,100,16
def remove_time_transpose_reshape(set,params):
    set=set.drop(columns=['t_axis'])
    set=set.transpose()
    set=set.values
    return set.reshape(params[0], params[1], params[2])

In [4]:
all_trains=[]
all_real=[]

for i in range(16):
    exec(f'all_trains.append(training_data["channel_{i+1}"][:400])')

for i in range(16):
    exec(f'all_real.append(real_data["channel_{i+1}"][:400])')

In [5]:
""" reshapes the data st it can fit in the CNN """
#print(type(training_data))
reshaped_data=remove_time_transpose_reshape(training_data,Params)
reshaped_real=remove_time_transpose_reshape(real_data,Params)
reshaped_test_data=remove_time_transpose_reshape(noisy_test,Params1)
reshaped_test_real=remove_time_transpose_reshape(real_test,Params1)


In [6]:
t=[]
for i in range(100):
   t.append(i)
t_axis=np.array(t)
print(t_axis)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]


In [7]:
"""normalize to go from -1 to 1"""
reshaped_data=reshaped_data/np.max(np.abs(reshaped_data))
reshaped_real=reshaped_real/np.max(np.abs(reshaped_real))
reshaped_test_data=reshaped_test_data/np.max(np.abs(reshaped_test_data))
reshaped_test_real=reshaped_test_real/np.max(np.abs(reshaped_test_real))

In [8]:
"""
def build_denoising_model(input_shape):
    model = models.Sequential()

    # Encoder
    model.add(layers.Conv1D(64, kernel_size=3, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling1D(pool_size=2, padding='same'))
    model.add(layers.Conv1D(128, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2, padding='same'))
    model.add(layers.Conv1D(256, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2, padding='same'))

    # Decoder
    model.add(layers.Conv1D(256, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.UpSampling1D(size=2))
    model.add(layers.Conv1D(128, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.UpSampling1D(size=2))
    model.add(layers.Conv1D(64, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.UpSampling1D(size=2))
    model.add(layers.Conv1D(input_shape[-1], kernel_size=3, activation='sigmoid', padding='same'))

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Example input shape: (time_steps, 16)
input_shape = (None, 16)  # None for variable time steps

# Build and compile the model
model = build_denoising_model(input_shape)
#model.summary()
"""

"\ndef build_denoising_model(input_shape):\n    model = models.Sequential()\n\n    # Encoder\n    model.add(layers.Conv1D(64, kernel_size=3, activation='relu', padding='same', input_shape=input_shape))\n    model.add(layers.MaxPooling1D(pool_size=2, padding='same'))\n    model.add(layers.Conv1D(128, kernel_size=3, activation='relu', padding='same'))\n    model.add(layers.MaxPooling1D(pool_size=2, padding='same'))\n    model.add(layers.Conv1D(256, kernel_size=3, activation='relu', padding='same'))\n    model.add(layers.MaxPooling1D(pool_size=2, padding='same'))\n\n    # Decoder\n    model.add(layers.Conv1D(256, kernel_size=3, activation='relu', padding='same'))\n    model.add(layers.UpSampling1D(size=2))\n    model.add(layers.Conv1D(128, kernel_size=3, activation='relu', padding='same'))\n    model.add(layers.UpSampling1D(size=2))\n    model.add(layers.Conv1D(64, kernel_size=3, activation='relu', padding='same'))\n    model.add(layers.UpSampling1D(size=2))\n    model.add(layers.Conv1D(i

In [9]:
def build_denoising_model(input_shape):
    model = models.Sequential()

    # Encoder
    model.add(layers.Conv1D(64, kernel_size=3, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling1D(pool_size=2, padding='same'))
    model.add(layers.Conv1D(128, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2, padding='same'))
    model.add(layers.Conv1D(256, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2, padding='same'))

    # Decoder
    model.add(layers.Conv1D(256, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.UpSampling1D(size=2))
    model.add(layers.Conv1D(128, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.UpSampling1D(size=2))
    model.add(layers.Conv1D(64, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.UpSampling1D(size=2))
    model.add(layers.Conv1D(input_shape[-1], kernel_size=3, activation='sigmoid', padding='same'))

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Example input shape: (time_steps, 16)
input_shape = (None, 16)  # None for variable time steps

# Build and compile the model
model = build_denoising_model(input_shape)
model.summary()

/home/shams/.virtualenvs/Research with Dr Ilya/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 300, 64)        │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 150, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 150, 128)       │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 75, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 75, 256)        │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 38, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 38, 256)        │       196,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling1d (UpSampling1D)    │ (None, 76, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 76, 128)        │        98,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling1d_1 (UpSampling1D)  │ (None, 152, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 152, 64)        │        24,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling1d_2 (UpSampling1D)  │ (None, 304, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 304, 16)        │         3,088 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 449,424 (1.71 MB)

 Trainable params: 449,424 (1.71 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:

# Train the model
model.fit(reshaped_data,reshaped_real , epochs=20, batch_size=1, validation_split=0.2)
model.save("CNN_model_2_todelete.keras")
print("Done!")


Epoch 1/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.0563 - val_loss: 0.0037
Epoch 2/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0036 - val_loss: 0.0037
Epoch 3/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0038 - val_loss: 0.0037
Epoch 4/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0037 - val_loss: 0.0037
Epoch 5/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0037 - val_loss: 0.0037
Epoch 6/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0038 - val_loss: 0.0037
Epoch 7/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0039 - val_loss: 0.0037
Epoch 8/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0038 - val_loss: 0.0037
Epoch 9/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0038 - val_loss: 0.0037
Epoch 10/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0038 - val_loss: 0.0037
Epoch 11/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0037 - val_loss: 0.0037
Epoch 12/100
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step

KeyboardInterrupt: 

In [ ]:
# Step 4: Evaluate and Visualize the Results
# Generate clean and noisy test waveforms
clean_test_waveform = reshaped_test_real
noisy_test_waveform = reshaped_test_data

print(np.shape(clean_test_waveform))
print(np.shape(noisy_test_waveform))


cleaned_test_waveform = model.predict(reshaped_test_data)
cleaned_after_trained= model.predict( reshaped_data)
print(np.shape(cleaned_test_waveform))
print(np.shape(cleaned_after_trained))


print(np.shape(all_trains))
print(np.shape(all_real))


In [ ]:
y_s_clean=[]
y_s_cleaned=[]
channel=1

for i in range(100):
    y_s_cleaned.append(cleaned_test_waveform[0][i][channel])
    y_s_clean.append(clean_test_waveform[0][i][channel])

y_s_clean=np.array(y_s_clean)
y_s_cleaned=np.array(y_s_cleaned)


In [ ]:
# Flatten the cleaned waveform
cleaned_test_waveform = cleaned_test_waveform.flatten()
real_test_waveform = reshaped_test_real.flatten()

cleaned_test_waveform=np.reshape(cleaned_test_waveform,(100,100,16))
real_test_waveform=np.reshape(real_test_waveform,(100,100,16))

In [ ]:
def display_n_plots_format2(data,n,m):
    print("--------------------------------------------------------")
    for i in range(n):
        exec(f'Channel{i+1}=np.array([])')
    for i in range(100):
        for j in range(n):
            exec(f'Channel{j+1}=np.append(Channel{j+1}, data[i][m][j])')
    Channels=[]
    for i in range(n):
        exec (f'Channels.append(Channel{i+1})')

    t_axis=np.arange(0,100,1)
    #exec(f'print(Channel{5})')
    exec(f'print(len(Channel{5}))')
    print(len(t_axis))

    fig, axs = plt.subplots(4, 4, figsize=(12, 12))  # Adjust the figure size
    y_limits = (-250, 250)
    for i, ax in enumerate(axs.flat):
        ax.plot(t_axis, Channels[i], label=f'Channel {i + 1}')

        ax.set_title(f'Channel {i + 1}', fontsize=10)
        ax.set_ylim(y_limits)
        ax.grid(True, lw=0.5)
        ax.label_outer()  # Only show outer labels to avoid redundancy


    fig.suptitle('16 Controlled channels', fontsize=22)  # Add a super title
    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to fit super title
    #plt.savefig(f'Figure {int(t_axis[-1] //200)+1} w 16 Controlled channels')
    plt.show()

print("Done!")

In [ ]:
print(np.shape(clean_test_waveform))
print(clean_test_waveform[0][1])
display_n_plots_format2(reshaped_real ,16,3)

In [ ]:
# Plotting the loss curves
plt.figure(figsize=(10, 5))
plt.plot( t_axis,y_s_clean , label='clean')
plt.plot(t_axis , y_s_cleaned, label='cleaned')
plt.title('Loss Curves')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()
print("Done!")